In [1]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [2]:
%reload_kedro

2022-08-04 10:08:53,654 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-04 10:08:53,700 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-04 10:08:54,268 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-04 10:08:54,269 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-04 10:08:54,278 - root - INFO - Registered line magic `run_viz`
2022-08-04 10:08:54,279 - root - INFO - Registered line magic `reload_kedro_mlflow`


In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [5]:
catalog.list()

['articles',
 'customers',
 'transactions',
 'customers_sample',
 'transactions_sample',
 'articles_sample',
 'input_images',
 'resized_images',
 'image_embeddings',
 'text_embeddings',
 'candidates',
 'train_transactions',
 'val_transactions',
 'train_candidates',
 'parameters',
 'params:image_embeddings_inference',
 'params:image_embeddings_inference.run_id',
 'params:image_embeddings_inference.image_path',
 'params:image_embeddings_inference.batch_size',
 'params:candidate_generation',
 'params:candidate_generation.global_articles',
 'params:candidate_generation.global_articles.n_days',
 'params:candidate_generation.global_articles.top_n',
 'params:candidate_generation.segment_articles',
 'params:candidate_generation.segment_articles.no_segments',
 'params:candidate_generation.segment_articles.n_days',
 'params:candidate_generation.segment_articles.top_n',
 'params:candidate_generation.image_embeddings',
 'params:candidate_generation.image_embeddings.n_last_bought',
 'params:candida

In [6]:
candidates = context.catalog.load('train_candidates')

2022-08-04 10:08:58,410 - kedro.io.data_catalog - INFO - Loading data from `train_candidates` (ParquetDataSet)...


In [7]:
val_transactions = context.catalog.load('val_transactions')

2022-08-04 09:30:34,563 - kedro.io.data_catalog - INFO - Loading data from `val_transactions` (CSVDataSet)...


In [9]:
val_transactions.shape, candidates.shape

((4461, 5), (137198, 7))

In [9]:
candidates.head(3)

,customer_id,global_articles,segment_articles,previously_bought,previously_bought_prod_name,closest_image_embeddings,closest_text_embeddings
0,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,"[0757995009, 0769748013, 0879248001, 081183500...","[0806451001, 0769748013, 0879248001, 081183500...",None,None,None,None
1,136601b1d1f36b13b9a4bbe185f164f24f522c5d476d09...,"[0757995009, 0769748013, 0879248001, 081183500...","[0490176004, 0769748013, 0879248001, 081183500...",None,None,None,None
2,247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d...,"[0757995009, 0769748013, 0879248001, 081183500...","[0793012001, 0879248001, 0930380003, 072490500...",None,None,None,None


In [10]:
val_transactions = val_transactions[['customer_id', 'article_id']]

In [11]:
val_transactions.drop_duplicates(inplace=True)

In [12]:
val_transactions.shape

(3979, 2)

In [13]:
val_transactions

,customer_id,article_id
0,017f1c01901a5113dffb00e2ff99024a99915d3cd86af4...,0916256001
1,01a39571fd15b542ac4832adcd4b80962823ee855a398a...,0855706004
2,02c8d36c490fc65c03d720c1220694573c34006cdcc2f9...,0903473003
3,0329f6a6682ad231723108c556d0861d8ff8a2fc5d843e...,0946764002
4,051aa83b1a3978fa888ae1d5a05e7eccceff95e00f7d7f...,0863692003
...,...,...
4456,fea6572677bb4067304af23467abc83e47c3926d35f625...,0906705001
4457,ff411f623177338bde6299009234da653cd417a6986780...,0903096002
4458,ff411f623177338bde6299009234da653cd417a6986780...,0923569007
4459,ff411f623177338bde6299009234da653cd417a6986780...,0873274001


In [14]:
col_list = [col for col in list(candidates.columns) if col != 'customer_id']

In [15]:
col_list

['global_articles',
 'segment_articles',
 'previously_bought',
 'previously_bought_prod_name',
 'closest_image_embeddings',
 'closest_text_embeddings']

I don't think that is efficient - for worst case scenario - ALL CUSTOMERS x 500

In [16]:
join_df = candidates[['customer_id', 'global_articles']].explode('global_articles')

In [17]:
join_df['dummy'] = 1

In [18]:
first = val_transactions.merge(join_df,
                       left_on=['customer_id', 'article_id'], right_on=['customer_id', 'global_articles'],
                       how='left')

In [19]:
first['dummy'] = first['dummy'].fillna(0)

In [20]:
first.dummy.value_counts()

0.0000    2482
1.0000    1497
Name: dummy, dtype: int64

In [21]:
first['dummy'].mean()

0.3762251822065846

In [22]:
candidates['global_articles']

0         [0757995009, 0769748013, 0879248001, 081183500...
1         [0757995009, 0769748013, 0879248001, 081183500...
2         [0757995009, 0769748013, 0879248001, 081183500...
3         [0757995009, 0769748013, 0879248001, 081183500...
4         [0757995009, 0769748013, 0879248001, 081183500...
                                ...                        
137193    [0757995009, 0769748013, 0879248001, 081183500...
137194    [0757995009, 0769748013, 0879248001, 081183500...
137195    [0757995009, 0769748013, 0879248001, 081183500...
137196    [0757995009, 0769748013, 0879248001, 081183500...
137197    [0757995009, 0769748013, 0879248001, 081183500...
Name: global_articles, Length: 137198, dtype: object

In [33]:
candidates.head()

,customer_id,global_articles,segment_articles,previously_bought,previously_bought_prod_name,closest_image_embeddings,closest_text_embeddings
0,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,"[0757995009, 0769748013, 0879248001, 081183500...","[0806451001, 0769748013, 0879248001, 081183500...",None,None,None,None
1,136601b1d1f36b13b9a4bbe185f164f24f522c5d476d09...,"[0757995009, 0769748013, 0879248001, 081183500...","[0490176004, 0769748013, 0879248001, 081183500...",None,None,None,None
2,247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d...,"[0757995009, 0769748013, 0879248001, 081183500...","[0793012001, 0879248001, 0930380003, 072490500...",None,None,None,None
3,d1e601dc9ea370b6dee95cbb983adf7fe5294386410be9...,"[0757995009, 0769748013, 0879248001, 081183500...","[0757995009, 0867044001, 0879248001, 076974801...",None,None,None,None
4,abc428504037054226353c4194380c7f16aff32b0633ac...,"[0757995009, 0769748013, 0879248001, 081183500...","[0757995009, 0867044001, 0879248001, 076974801...",None,None,None,None


In [64]:
candidates.columns

Index(['customer_id', 'global_articles', 'segment_articles',
       'previously_bought', 'previously_bought_prod_name',
       'closest_image_embeddings', 'closest_text_embeddings'],
      dtype='object')

In [60]:
def add_lists_or_none(*args):
    empty_set = set()
    for arg in args:
        if arg is None:
            continue
        new_item = set(arg)
        empty_set |= new_item
    return list(empty_set)

In [73]:
mylist = ['global_articles', 'segment_articles',
       'previously_bought', 'previously_bought_prod_name',
       'closest_image_embeddings', 'closest_text_embeddings']

In [75]:
results = 

In [ ]:
candidates.apply(lambda x: *)

In [61]:
candidates['all_candidates'] = candidates.apply(lambda x: add_lists_or_none(x.global_articles,
                                             x.segment_articles,
                                             x.previously_bought,
                                             x.previously_bought_prod_name,
                                             x.closest_image_embeddings,
                                             x.closest_text_embeddings
                                            ), axis=1)

In [67]:
candidates.loc[137193,:]

customer_id                    a4092d95d91ec0ff90527962935550346d993a299353c6...
global_articles                [0757995009, 0769748013, 0879248001, 081183500...
segment_articles               [0757995009, 0867044001, 0879248001, 076974801...
previously_bought                                                   [0870765002]
previously_bought_prod_name                 [0870765002, 0702933008, 0870765004]
closest_image_embeddings        [0721390005, 0607367012, 0547780046, 0737948002]
closest_text_embeddings         [0870765004, 0810594001, 0859987001, 0779930001]
all_candidates                 [0843793004, 0883026001, 0846181003, 075987100...
Name: 137193, dtype: object

In [68]:
len(candidates.loc[137193,:].global_articles)

248

In [69]:
len(candidates.loc[137193,:].segment_articles)

246

In [72]:
len(candidates.loc[137193,:].closest_text_embeddings)

4

In [71]:
len(candidates.loc[137193,:].all_candidates)

317

In [ ]:
df['E']=(df.B+df.C).map(set).map(list)

pandas + looping = inefficient too :D :D :D

In [40]:
val = val_transactions.groupby(['customer_id'])['article_id'].apply(list).reset_index()

In [41]:
val.head()

,customer_id,article_id
0,00459e5341c64d459a496c4cf5eb9016e2df3a34cbafaf...,[0832307007]
1,00462904b288681a9facb555f75dd2cf4d0f730a6e0ea7...,"[0827968001, 0874819002, 0910457001]"
2,00a69bed783860d6d82da6f451b20892f7e1355c241c0b...,"[0730683062, 0862271003]"
3,00bce12709bdc1536ee25bc63e229d3746960b20cbbb75...,[0869331006]
4,00c571917952c5c99ffb38e7f75b2e91a6b179591cd030...,"[0752689002, 0843793004, 0505882002]"


In [43]:
val.shape

(2865, 2)

In [44]:
val = val.merge(candidates, on='customer_id', how='left')

In [45]:
val.head()

,customer_id,article_id,global_articles,segment_articles,previously_bought,previously_bought_prod_name,closest_image_embeddings,closest_text_embeddings
0,00459e5341c64d459a496c4cf5eb9016e2df3a34cbafaf...,[0832307007],"[0757995009, 0769748013, 0879248001, 081183500...","[0806451001, 0769748013, 0879248001, 081183500...","[0817484005, 0839284002, 0532123001, 083231100...","[0835008005, 0707269001, 0707269011, 087367900...","[0678924003, 0921226002, 0877961022, 092151900...","[0832481003, 0832481001, 0832481005, 077455200..."
1,00462904b288681a9facb555f75dd2cf4d0f730a6e0ea7...,"[0827968001, 0874819002, 0910457001]","[0757995009, 0769748013, 0879248001, 081183500...","[0757995009, 0867044001, 0879248001, 076974801...",[0903924001],"[0903924002, 0903924001]","[0814879002, 0765773004, 0800437002, 0711280006]","[0903924002, 0910961001, 0915444001, 0900183001]"
2,00a69bed783860d6d82da6f451b20892f7e1355c241c0b...,"[0730683062, 0862271003]","[0757995009, 0769748013, 0879248001, 081183500...","[0757995009, 0867044001, 0879248001, 076974801...","[0853556001, 0844160002, 0918525001]","[0818030001, 0853556001, 0844160002, 0918525001]","[0544515005, 0746344002, 0354246017, 089126500...","[0918525004, 0924320002, 0924320001, 090758500..."
3,00bce12709bdc1536ee25bc63e229d3746960b20cbbb75...,[0869331006],"[0757995009, 0769748013, 0879248001, 081183500...","[0757995009, 0867044001, 0879248001, 076974801...","[0599580047, 0811835001, 0864216003]","[0599580074, 0599580038, 0599580047, 059958005...","[0609357004, 0902486002, 0785791001, 075662400...","[0864216001, 0864216017, 0864216004, 086421600..."
4,00c571917952c5c99ffb38e7f75b2e91a6b179591cd030...,"[0752689002, 0843793004, 0505882002]","[0757995009, 0769748013, 0879248001, 081183500...","[0757995009, 0867044001, 0879248001, 076974801...","[0739590030, 0802979002, 0599580047, 055960101...","[0776237034, 0776237020, 0589599031, 077623701...","[0845445001, 0730683019, 0661888004, 081190500...","[0880839006, 0880839002, 0880839003, 088304100..."


In [8]:
def _concatenate_all_lists(*args):
    articles_set = set()
    for arg in args:
        if arg is None:
            continue
        new_items = set(arg)
        articles_set |= new_items
    return list(articles_set) 

In [9]:
candidates

,customer_id,global_articles,segment_articles,previously_bought,previously_bought_prod_name,closest_image_embeddings,closest_text_embeddings
0,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0897687001, 045616308...",None,None,None,None
1,136601b1d1f36b13b9a4bbe185f164f24f522c5d476d09...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0750423025, 085024100...",None,None,None,None
2,247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0750423025, 085024100...",None,None,None,None
3,d1e601dc9ea370b6dee95cbb983adf7fe5294386410be9...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0854677004, 089465100...",None,None,None,None
4,abc428504037054226353c4194380c7f16aff32b0633ac...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0897687001, 045616308...",None,None,None,None
...,...,...,...,...,...,...,...
137193,a4092d95d91ec0ff90527962935550346d993a299353c6...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0897687001, 045616308...",[0870765002],"[0870765002, 0702933008, 0870765004]","[0721390005, 0607367012, 0547780046, 0737948002]","[0870765004, 0810594001, 0859987001, 0779930001]"
137194,c321175e12787427aa6d00985519c41650614b362e206f...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0854677004, 089465100...",None,None,None,None
137195,016a81548c36c010144819354910dc0763b9b10eeeeacf...,"[0717490010, 0903926001, 0456163086, 090595700...","[0889828001, 0717490010, 0903926001, 085024100...",[0904422001],[0904422001],"[0650388001, 0711138001, 0485544020, 0560209005]","[0933032002, 0933032001, 0933032003, 0806735001]"
137196,dc066cb0d1be0a395c472b1a31679fcd64035db1a42121...,"[0717490010, 0903926001, 0456163086, 090595700...","[0717490010, 0903926001, 0897687001, 045616308...",None,None,None,None


In [60]:
aa = [col for col in candidates.columns if col != 'customer_id']

In [39]:
candidates.global_articles[0]

array(['0717490010', '0903926001', '0456163086', '0905957007',
       '0904422001', '0874113026', '0552716001', '0896343002',
       '0918525001', '0898573003', '0816563002', '0853881001',
       '0763469002', '0759814009', '0935840001', '0908081003',
       '0832298006', '0706016015', '0873279001', '0852174001',
       '0678942054', '0759871001', '0882888002', '0698276009',
       '0816423002', '0748355002', '0867969002', '0860819001',
       '0895002004', '0855706004', '0811835009', '0850917003',
       '0879088001', '0918894001', '0898692006', '0865929002',
       '0817353008', '0933706001', '0880839001', '0883307004',
       '0399256005', '0556255001', '0913688003', '0685813042',
       '0832473001', '0776237020', '0571706001', '0677930066',
       '0817477001', '0572998001', '0873279003', '0916468002',
       '0923037002', '0869331006', '0811907003', '0926500001',
       '0874819003', '0909916001', '0879248001', '0599580047',
       '0212629048', '0783346020', '0824352001', '09421

In [67]:
aa

['global_articles',
 'segment_articles',
 'previously_bought',
 'previously_bought_prod_name',
 'closest_image_embeddings',
 'closest_text_embeddings']

In [93]:
candidates[aa].values.shape

(137198, 6)

In [137]:
aa + ['all_candidates']

['global_articles',
 'segment_articles',
 'previously_bought',
 'previously_bought_prod_name',
 'closest_image_embeddings',
 'closest_text_embeddings',
 'all_candidates']

In [142]:
def add_lists(x, number):
    empty_set = set()
    for i in range(number):
        if x[i] is None:
            continue
        empty_set |= set(x[i])
    return empty_set

In [143]:
candidates['all_candidssates'] = np.apply_along_axis(add_lists, axis=1, arr=candidates[aa].values, number=6)

In [ ]:
candidates.apply(lambda x: _concatenate_all_lists(x.global_articles,
                                             x.segment_articles,
                                             x.previously_bought,
                                             x.previously_bought_prod_name,
                                             x.closest_image_embeddings,
                                             x.closest_text_embeddings
                                            ), axis=1)

In [130]:
candidates['all_candidates_proper'] = candidates.apply(lambda x: _concatenate_all_lists(x.global_articles,
                                             x.segment_articles,
                                             x.previously_bought,
                                             x.previously_bought_prod_name,
                                             x.closest_image_embeddings,
                                             x.closest_text_embeddings
                                            ), axis=1)

In [135]:
(candidates.all_candidates.apply(list)==candidates.all_candidates_proper).mean()

1.0